In [1]:
import pandas as pd
import numpy as np

In [2]:
database_type = "/full"

The `lab` table provides Laboratory tests that have have been mapped to a standard set of measurements. 
Unmapped measurements are recorded in the customLab table.
See: http://eicu-crd.mit.edu/eicutables/lab/

#### Reading in lab dataset and keeping relevant columns to our study.

In [3]:
columns = ['patientunitstayid', 'labresultoffset', 'labname', 'labresult']
lab = pd.read_csv('../../eICU' + database_type + '/lab.csv', usecols=columns)
lab

,patientunitstayid,labresultoffset,labname,labresult
0,141168,2026,fibrinogen,177.00
1,141168,1133,PT - INR,2.50
2,141168,2026,magnesium,2.00
3,141168,1133,PT,26.60
4,141168,2141,pH,7.20
5,141168,231,PT - INR,1.70
6,141168,1701,urinary creatinine,173.12
7,141168,2026,MCH,29.20
8,141168,516,BUN,26.00
9,141168,1701,urinary sodium,12.00


#### Keeping lab results relevant to diagnosing sepsis

In [4]:
# converting labname 'urinary creatinine to 'creatinine'
lab.loc[lab['labname'].str.contains("urinary creatinine"), 'labname'] = "creatinine"

# Only keeping rows where labname is 'WBC x 1000', 'lactate', 'creatinine'
lab_name = ['WBC x 1000', 'lactate', 'creatinine']
lab = lab[lab.labname.isin(lab_name)]

#### Transposing Lab Names and their results into columns.

In [5]:
# duplicating labs to transpose labresult and labname
labc = lab.copy()
labw = lab.copy()
labl = lab.copy()

labc = labc.loc[(labc['labname'].str.contains("creatinine"))]
labw = labw.loc[(labw['labname'].str.contains("WBC x 1000"))]
labl = labl.loc[(labl['labname'].str.contains("lactate"))]

# Transpose labresults and lab name to cols
labc['creatinine'] = labc.apply(lambda x: x['labresult'] if x['labname'] == 'creatinine' else -1, axis=1)
labw['wbcx1000'] = labw.apply(lambda x: x['labresult'] if x['labname'] == 'WBC x 1000' else -1, axis=1)
labl['lactate'] = labl.apply(lambda x: x['labresult'] if x['labname'] == 'lactate' else -1, axis=1)

labc = labc.drop(['labname', 'labresult'], axis = 1)
labw = labw.drop(['labname', 'labresult'], axis = 1)
labl = labl.drop(['labname', 'labresult'], axis = 1)

#### Merging Lab Dataframes into Final Lab Dataset

In [6]:
# merge lab results dataframes
finallab = labc.merge(labw, how = 'left', on = ['patientunitstayid', 'labresultoffset'])
finallab = finallab.merge(labl, how = 'left', on = ['patientunitstayid', 'labresultoffset'])
finallab = finallab.drop_duplicates()
finallab

,patientunitstayid,labresultoffset,creatinine,wbcx1000,lactate
0,141168,1701,173.12,NaN,NaN
1,141168,2026,2.95,19.8,12.2
2,141168,1133,2.30,14.7,NaN
3,141168,516,1.95,9.8,NaN
4,141178,-280,0.70,7.6,NaN
5,141179,82,0.70,NaN,NaN
6,141179,1487,0.70,8.1,NaN
7,141179,750,0.70,NaN,NaN
8,141194,11658,1.75,8.9,NaN
9,141194,10397,1.74,8.7,NaN


#### Saving the Cleaned Patient DataFrame to a `csv` file

In [7]:
path = "../../eICU/training/"
finallab.to_csv(path + "lab.csv", sep=',', index=False, encoding='utf-8')